# Auto refering

In [1]:
import json
import random
import nltk.data
import numpy as np

from tqdm import tqdm
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from nltk.tokenize import sent_tokenize
from nltk.tokenize import RegexpTokenizer, WordPunctTokenizer

## Args

In [2]:
# INPUT = 'example_texts.json'
INPUT = 'dataset_43428_1.txt'
OUTPUT = 'referats.json'

## Read & Analysis

In [3]:
texts = json.load(open(INPUT, 'r'))
SAMPLE_TEXT = texts[0]
SAMPLE_TEXT

'А бойтесь единственно только того , кто скажет :\n« Я знаю , как надо ! »\nСтивен Хант использует стилистику « парового панка » , для того чтобы поговорить об актуальных проблемах современности\n\nСтимпанк , он же « паровой панк » , — это прежде всего антураж : субмарины на паровом ходу , аэростаты , наполненные летучим газом , дамы в кринолинах и мужчины с тросточками .\nА ещё это атмосфера : затхлая безнадёга работных домов , где гнут спины юные Оливеры Твисты , готика сельских кладбищ , скрежет шестерёнок и кривошипов в недрах гигантских дифференциальных исчислителей ...\nКонан Дойл пополам с Диккенсом плюс капелька кафкианского абсурда и лавкрафтовского ужаса : викторианский мир , резко шагнувший в эпоху НТР , но не преодолевший противоречия , разрывающие сословное общество изнутри .\nВсё это есть в романе Стивена Ханта , а также подземный город , основанный изгоями , живущими надеждой на пролетарскую революцию , разумные роботы - паровики , мутанты , наделённые сверхъестественным

In [4]:
max(len(text) for text in texts)

24694

In [5]:
sum(len(text) for text in texts) / len(texts)

3027.31

## Referers

In [6]:
sent_tokenize(SAMPLE_TEXT)[0]

'А бойтесь единственно только того , кто скажет :\n« Я знаю , как надо !'

In [7]:
def get_referat_sent(text):
    return sent_tokenize(text)[0:2]

In [8]:
def get_referat_prefix(text):
    return text[:300]

In [9]:
class SimpleSummarizer:
    def summarize(self, input, num_sentences=3):
        return " ".join(self._get_summarized(input, num_sentences))
    
    def _reorder_sentences(self, output_sentences, input):
        output_sentences.sort(key=input.find)
        return output_sentences
    
    def _get_summarized(self, input, num_sentences):
        tokenizer = RegexpTokenizer('\w+')
        
        # get the frequency of each word in the input
        base_words = [word.lower() for word in tokenizer.tokenize(input)]
        words = [word for word in base_words if word not in stopwords.words()]
        word_frequencies = FreqDist(words)
        
        # now create a set of the most frequent words
        most_frequent_words = [w for _, w in sorted((c, w) for w, c in word_frequencies.items())[:100]]
        
        # break the input up into sentences.  working_sentences is used
        # for the analysis, but actual_sentences is used in the results
        # so capitalization will be correct.
        
        sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')
        actual_sentences = sent_detector.tokenize(input)
        working_sentences = [sentence.lower() for sentence in actual_sentences]
        
        # iterate over the most frequent words, and add the first sentence
        # that inclues each word to the result.
        output_sentences = []
        for word in most_frequent_words:
            for i in range(0, len(working_sentences)):
                if (word in working_sentences[i] and actual_sentences[i] not in output_sentences):
                    output_sentences.append(actual_sentences[i])
                    break
                if len(output_sentences) >= num_sentences: break
            if len(output_sentences) >= num_sentences: break
            
        # sort the output sentences back to their original order
        return self._reorder_sentences(output_sentences, input)

In [10]:
import re, pdb, sys, math
from collections import defaultdict


class Graph:
	def __init__(self):
		self.Vertices = []
		self.Edges = []

	def getRankedVertices(self):
		res = defaultdict(float)
		for e in self.Edges:
			res[e.Vertex1] += e.Weight
		return sorted(res.items(), key=lambda x: x[1], reverse=True)

class Vertex:
	def __init__(self):
		self.Sentence = None

class Edge:
	def __init__(self):
		self.Vertex1 = None
		self.Vertex2 = None
		self.Weight = 0

class WordType:
	Content=0
	Function=1
	ContentPunctuation=2
	FunctionPunctuation=3

class Word:
	def __init__(self):
		self.Text=''
		self.Type=''

class Sentence:
	def __init__(self):
		self.Words = []

	def getFullSentence(self):
		text = ''
		for w in self.Words:
			text += w.Text
		return text.strip()

	def getReducedSentence(self):
		sentenceText = ''
		sentenceEnd = self.Words[len(self.Words)-1]
		contentWords = list(filter(lambda w: w.Type == WordType.Content, self.Words))
		i = 0
		while i < len(contentWords):
			w = contentWords[i]
			# upper case the first character of the sentence
			if i == 0:
				li = list(w.Text)
				li[0] = li[0].upper()
				w.Text = ''.join(li)
			sentenceText += w.Text
			if i < len(contentWords)-1:
				sentenceText += ' '
			elif sentenceEnd.Text != w.Text:
				sentenceText += sentenceEnd.Text
			i = i+1
		return sentenceText
			

class Paragraph:
	def __init__(self):
		self.Sentences = []

class Reduction:
	functionPunctuation = ' ,-'
	contentPunctuation = '.?!\n'
	punctuationCharacters = functionPunctuation+contentPunctuation
	sentenceEndCharacters = '.?!'
	
	def isContentPunctuation(self, text):
		for c in self.contentPunctuation:
			if text.lower() == c.lower():
				return True
		return False

	def isFunctionPunctuation(self, text):
		for c in self.functionPunctuation:
			if text.lower() == c.lower():
				return True
		return False

	def isFunction(self, text, stopWords):
		for w in stopWords:
			if text.lower() == w.lower():
				return True
		return False

	def tag(self, sampleWords, stopWords):
		taggedWords = []
		for w in sampleWords:
			tw = Word()
			tw.Text = w
			if self.isContentPunctuation(w):
				tw.Type = WordType.ContentPunctuation
			elif self.isFunctionPunctuation(w):
				tw.Type = WordType.FunctionPunctuation
			elif self.isFunction(w, stopWords):
				tw.Type = WordType.Function
			else:
				tw.Type = WordType.Content
			taggedWords.append(tw)
		return taggedWords

	def tokenize(self, text):
		return list(filter(lambda w: w != '', re.split('([{0}])'.format(self.punctuationCharacters), text)))

	def getWords(self, sentenceText, stopWords):
		return self.tag(self.tokenize(sentenceText), stopWords) 

	def getSentences(self, line, stopWords):
		sentences = []
		sentenceTexts = list(filter(lambda w: w.strip() != '', re.split('[{0}]'.format(self.sentenceEndCharacters), line)))
		sentenceEnds = re.findall('[{0}]'.format(self.sentenceEndCharacters), line)
		sentenceEnds.reverse()
		for t in sentenceTexts:
			if len(sentenceEnds) > 0:
				t += sentenceEnds.pop()
			sentence = Sentence()
			sentence.Words = self.getWords(t, stopWords)
			sentences.append(sentence)
		return sentences

	def getParagraphs(self, lines, stopWords):
		paragraphs = []
		for line in lines:
			paragraph = Paragraph()
			paragraph.Sentences = self.getSentences(line, stopWords)
			paragraphs.append(paragraph)
		return paragraphs

	def findWeight(self, sentence1, sentence2):
		length1 = len(list(filter(lambda w: w.Type == WordType.Content, sentence1.Words)))
		length2 = len(list(filter(lambda w: w.Type == WordType.Content, sentence2.Words)))
		if length1 < 4 or length2 < 4:
			return 0
		weight = 0
		for w1 in list(filter(lambda w: w.Type == WordType.Content, sentence1.Words)):
			for w2 in list(filter(lambda w: w.Type == WordType.Content, sentence2.Words)):
				if w1.Text.lower() == w2.Text.lower():
					weight = weight + 1
		normalised1 = 0
		if length1 > 0:
			normalised1 = math.log(length1)
		normalised2 = 0
		if length2 > 0:
			normalised2 = math.log(length2)
		norm = normalised1 + normalised2
		if norm == 0:
			return 0
		return weight / float(norm)

	def buildGraph(self, sentences):
		g = Graph()
		for s in sentences:
			v = Vertex()
			v.Sentence = s
			g.Vertices.append(v)
		for i in g.Vertices:
			for j in g.Vertices:
				if i != j:
					w = self.findWeight(i.Sentence, j.Sentence)
					e = Edge()
					e.Vertex1 = i
					e.Vertex2 = j
					e.Weight = w
					g.Edges.append(e)
		return g

	def sentenceRank(self, paragraphs):
		sentences = []
		for p in paragraphs:
			for s in p.Sentences:
				sentences.append(s)
		g = self.buildGraph(sentences)
		return g.getRankedVertices()

	def reduce(self, text, reductionRatio):
		stopWords= stopwords.words()

		lines = text.splitlines()
		contentLines = list(filter(lambda w: w.strip() != '', lines))

		paragraphs = self.getParagraphs(contentLines, stopWords)

		rankedSentences = self.sentenceRank(paragraphs)

		orderedSentences = []
		for p in paragraphs:
			for s in p.Sentences:
				orderedSentences.append(s)

		reducedSentences = []
		i = 0
		while i < math.trunc(len(rankedSentences) * reductionRatio):
			s = rankedSentences[i][0].Sentence
			position = orderedSentences.index(s)
			reducedSentences.append((s, position))
			i = i + 1
		reducedSentences = sorted(reducedSentences, key=lambda x: x[1])
		
		reducedText = []
		for s,r in reducedSentences:
			reducedText.append(s.getFullSentence())
		return reducedText	

    
def reduct_reducer(text):
    n = len(text)
    if n <= 350:
        return text
    else:
        return ' '.join(Reduction().reduce(text, 350 / n))

In [11]:
from sumy.utils import get_stop_words
from sumy.nlp.stemmers import Stemmer
from sumy.parsers.plaintext import PlaintextParser
from sumy.summarizers.lsa import LsaSummarizer as Summarizer
from sumy.nlp.tokenizers import Tokenizer


def summarize_sumy(text):
    summarizer = Summarizer(Stemmer('russian'))
    summarizer.stop_words = stopwords.words()
    parser = PlaintextParser.from_string(text, Tokenizer('english'))
    return ' '.join(str(s) for s in summarizer(parser.document, 3))

In [104]:
from summa import summarizer


def summarize_summa(text):
    n = len(text)
    if n <= 350:
        return text
    else:
        return summarizer.summarize(text, ratio=350 / n, language='russian')

ModuleNotFoundError: No module named 'graph'

In [12]:
eval_referer = summarize_sumy

In [99]:
sample_text = random.sample(texts, 1)[0]
referencing = eval_referer(sample_text)
sample_text

'Митрополит Иларион призвал поклонников Сталина отрезвиться и съездить на Бутовский полигон\nПредседатель ОВЦС предложил тем , кто оправдывает сталинские репрессии , ответить на вопрос , за что расстреливали 15-летних детей , и заявил , что таким преступлениям не может быть никакого оправдания .\n\nГлава Отдела внешних церковных связей Московского патриархата , митрополит Волоколамский Иларион ( Алфеев ) gрокомментировал заявления социологов о росте популярности Иосифа Сталина у жителей России .\nПо его мнению , тем , кто считает Сталина выдающимся деятелем , следует &quot; отрезвиться &quot; .\nМитрополит Иларион предложил , оценивая роль Сталина в истории , учитывать два аспекта .\n&quot; Мы знаем , что есть историческая заслуга всех тех , кто со стороны нашего государства вел войну , чтобы мы победили , а не проиграли , и этой заслуги нельзя умалять .\nНо то , что были многомиллионные жертвы , репрессии , то , что был геноцид собственного населения , - это все было .\nМы не можем , 

In [100]:
referencing

'Глава Отдела внешних церковных связей Московского патриархата , митрополит Волоколамский Иларион ( Алфеев ) gрокомментировал заявления социологов о росте популярности Иосифа Сталина у жителей России . Митрополит Иларион также прокомментировал закон об оскорблении чувств верующих , отметив , что &quot; никто никому не должен запрещать свободу высказываний : если вам не нравится Церковь , ее учение или поведение отдельных членов , вы можете об этом говорить . Этот закон , по мнению митрополита Волоколамского , в России действительно необходим : &quot; Мы живем в особой стране , где на протяжении веков мирно сосуществуют разные религии и конфессии .'

In [101]:
len(referencing)

654

## Submission

In [16]:
json.dump([eval_referer(text) for text in tqdm(texts)], open(OUTPUT, 'w'))

100%|██████████| 200/200 [00:21<00:00,  9.10it/s]
